<a href="https://colab.research.google.com/github/patty1997/Deep_Learning_Projects/blob/main/Leaf_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_addons

In [ ]:
!cp gdrive/MyDrive/denseNet.py
import denseNet
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, LeakyReLU, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, ReLU,Activation, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import activations
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import numpy as np
import os
import cv2

cp: missing destination file operand after 'gdrive/MyDrive/denseNet.py'
Try 'cp --help' for more information.


In [ ]:
img_rows = 256
img_cols = 256
channels = 3
img_shape = (img_rows, img_cols, channels)

train_dir = "/content/drive/MyDrive/dataset/diseased/"

In [ ]:
def build_generator():
    
    noise_shape = (100,)
    
    model = Sequential();
    n_nodes = 128 * 1 * 1
    model.add(Dense(n_nodes,input_shape = noise_shape))
    model.add(Reshape((1,1,128)))
    
    model.add(Conv2DTranspose(2048, (8,8), strides = (4,4) , padding = 'same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(1024, (4,4), strides = (2,2) , padding = 'same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(512, (4,4), strides = (2,2) , padding = 'same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(256, (4,4), strides = (2,2) , padding = 'same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(128, (4,4), strides = (2,2) , padding = 'same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(64, (4,4), strides = (2,2) , padding = 'same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(3, (4,4), strides = (2,2) , padding = 'same'))
    model.add(Activation(activations.tanh))
    
    model.add(Reshape(img_shape))

    model.summary()
    
    noise = Input(shape = noise_shape)
    img = model(noise)
    
    return Model(noise,img)


In [ ]:
def build_discriminator():
    
    
        
    inputs = Input( shape = img_shape )
    x = Conv2D( 64 , kernel_size=( 4 , 4 ), strides=(2 , 2) , use_bias=False, padding = 'same', 
               kernel_initializer='he_normal' , kernel_regularizer=l2( 1e-4 ) )( inputs )
    x = tfa.layers.InstanceNormalization( gamma_initializer = "random_uniform" )(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x, num_filters = denseNet.dense_block(inputs=x, num_layers=6, num_filters=32, growth_rate=0, dropout_rate=0.2)
    x = denseNet.transition(inputs=x, compression_factor=0.5, dropout_rate=0.2)
    x,num_filters = denseNet.dense_block(inputs=x, num_layers=9, num_filters=32, growth_rate=0, dropout_rate=0.2)

    x = tfa.layers.InstanceNormalization( gamma_initializer = "random_uniform" )(x)
    x = Activation('relu')(x)
    #x = tfa.layers.AdaptiveAveragePooling2D((1,1))(x)
    x = Flatten()(x)    
    x = Dense(1,activation='sigmoid')(x)
    #model.summary()
    
    return Model(inputs, x)


In [ ]:
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return np.array(images)


In [ ]:
def train(epochs, batch_size = 128, save_interval = 500):
    
    X_train = load_images(train_dir);
    
    X_train = (X_train.astype(np.float32)-127.5) / 127.5
    
    half_batch = int(batch_size / 2)
    
    for epoch in range(epochs):
        
        #Train Discriminator
        
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
        
        noise = np.random.normal(0,1,(half_batch,100))
        print("Entered")
        gen_imgs = generator.predict(noise)
        
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        
        #Train generator
        
        noise = np.random.normal(0, 1, (batch_size,100))
        
        valid_y = np.array([1]*batch_size)
        
        g_loss = combined.train_on_batch(noise, valid_y)
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        
        if epoch % save_interval == 0:
            save_imgs(epoch)
            


In [ ]:
def save_imgs(epoch):
    
    r,c = 5, 5
    noise = np.random.normal(0,1,(r*c,100))
    gen_imgs = generator.predict(noise)
    
    gen_imgs = 0.5*gen_imgs + 0.5
    
    fig, axs = plt.subplots(r, c)
    
    cnt = 0
    
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0],cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()


In [ ]:
optimizer = Adam(0.0002, 0.5)

discriminator = build_discriminator()

discriminator.compile(loss = 'binary_crossentropy',
                      optimizer = optimizer,
                      metrics = ['accuracy'])

generator = build_generator()

generator.compile(loss = 'binary_crossentropy', optimizer = optimizer)

z = Input(shape = (100,))
img = generator(z)


discriminator.trainable = False

valid = discriminator(img)

combined = Model(z,valid)

combined.compile(loss = 'binary_crossentropy', optimizer = optimizer)

train(epochs = 100, batch_size = 256, save_interval = 10)

generator.save('generator_model.h5')


(None, 64, 64, 96)
(None, 64, 64, 128)
(None, 64, 64, 160)
(None, 64, 64, 192)
(None, 64, 64, 224)
(None, 64, 64, 256)
(None, 64, 64, 256)
(None, 32, 32, 160)
(None, 32, 32, 192)
(None, 32, 32, 224)
(None, 32, 32, 256)
(None, 32, 32, 288)
(None, 32, 32, 320)
(None, 32, 32, 352)
(None, 32, 32, 384)
(None, 32, 32, 416)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               12928     
_________________________________________________________________
reshape_4 (Reshape)          (None, 1, 1, 128)         0         
_________________________________________________________________
conv2d_transpose_14 (Conv2DT (None, 4, 4, 2048)        16779264  
_________________________________________________________________
batch_normalization_12 (Batc (None, 4, 4, 2048)        8192      
_________________________________________________________________

ResourceExhaustedError: ignored